In [3]:
import numpy as np

In [4]:
def quaternion_rotation_matrix(Q, T):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
    
    tx = T[0]
    ty = T[1]
    tz = T[2]
    
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
    
    translation_matrix = np.array([[tx],
                                   [ty],
                                   [tz]])
                            
    return rot_matrix, translation_matrix

In [11]:
# [0.999624, -0.0232377, -0.00441006, 0.0138727], [0.0877391, -5.68903, -1.18716]
# [0.999578, -0.0241463, -0.0054248, 0.015219], [0.0443816, -5.26152, -1.21401]
R, T = quaternion_rotation_matrix([0.999578, -0.0241463, -0.0054248, 0.015219], [0.0443816, -5.26152, -1.21401])

In [18]:
## 2.950000000000000000e+02,3.700000000000000000e+01,4.600000000000000000e+02,1.900000000000000000e+02
## 2.770000000000000000e+02,6.500000000000000000e+01,4.870000000000000000e+02,2.550000000000000000e+02,6.930000000000000000e+02,6.700000000000000000e+01,8.930000000000000000e+02,2.500000000000000000e+02
sX, sY, eX, eY = 2.770000000000000000e+02,6.500000000000000000e+01,4.870000000000000000e+02,2.550000000000000000e+02
start3D = np.matmul(R, np.array([[sX], [sY], [0]])) + T
end3D = np.matmul(R, np.array([[eX], [eY], [0]])) + T
print('start3D:', start3D)
print('end3D:', end3D)

start3D: [[274.93930399]
 [ 68.13294465]
 [ -1.56195152]]
end3D: [[479.09877346]
 [264.26777304]
 [ -8.64193439]]


In [15]:
###
start3D: [[293.94412671]
 [ 39.49905013]
 [ -0.49987153]]
end3D: [[454.66211901]
 [196.88502064]
 [ -6.27826438]]

<>:2: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:5: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:2: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:5: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<ipython-input-15-69a970e65c5d>:2: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
  start3D: [[293.94412671]


TypeError: list indices must be integers or slices, not float